In [4]:
import pandas as pd
import numpy as np


# קריאת הקובץ שהעלינו
df = pd.read_csv('/content/pregnancies.csv')

# הצגת 5 שורות ראשונות
print(df.head())


   Pregnancies  Glucose  BloodPressure  High Insulin   BMI  \
0            6      148             72             0  33.6   
1            1       85             66             0  26.6   
2            8      183             64             0  23.3   
3            1       89             66             1  28.1   
4            0      137             40             1  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


In [9]:
def detect_outliers_std(df):
    outliers = pd.DataFrame()

    # עוברים על כל העמודות המספריות
    for col in df.select_dtypes(include=[np.number]).columns:
        mean = df[col].mean()
        std = df[col].std()

        print(f"\n🔵 עמודה: {col}")
        print(f"ממוצע: {mean:.2f}, סטיית תקן: {std:.2f}")

        # חיפוש ערכים חריגים: מחוץ ל-3 סטיות תקן
        outliers_in_col = df[(df[col] > mean + 3*std) | (df[col] < mean - 3*std)]
        print(f"➔ נמצאו {len(outliers_in_col)} חריגים בעמודה")

        outliers = pd.concat([outliers, outliers_in_col])

    # הסרת כפילויות אם יש
    return outliers.drop_duplicates()

# קריאה לפונקציה על הדאטה
outliers_std = detect_outliers_std(df)

print("\n🟠 כל השורות החריגות לפי סטיית תקן:")
display(outliers_std)


🔵 עמודה: Pregnancies
ממוצע: 3.85, סטיית תקן: 3.37
➔ נמצאו 4 חריגים בעמודה

🔵 עמודה: Glucose
ממוצע: 122.82, סטיית תקן: 56.68
➔ נמצאו 1 חריגים בעמודה

🔵 עמודה: BloodPressure
ממוצע: 69.11, סטיית תקן: 19.36
➔ נמצאו 35 חריגים בעמודה

🔵 עמודה: High Insulin
ממוצע: 0.56, סטיית תקן: 3.34
➔ נמצאו 1 חריגים בעמודה

🔵 עמודה: BMI
ממוצע: 32.04, סטיית תקן: 8.29
➔ נמצאו 17 חריגים בעמודה

🔵 עמודה: DiabetesPedigreeFunction
ממוצע: 0.47, סטיית תקן: 0.33
➔ נמצאו 11 חריגים בעמודה

🔵 עמודה: Age
ממוצע: 33.24, סטיית תקן: 11.76
➔ נמצאו 5 חריגים בעמודה

🔵 עמודה: Outcome
ממוצע: 0.35, סטיית תקן: 0.48
➔ נמצאו 0 חריגים בעמודה

🟠 כל השורות החריגות לפי סטיית תקן:


,Pregnancies,Glucose,BloodPressure,High Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
88,15,136,70,1,37.1,0.153,43,1
159,17,163,72,0,40.9,0.817,47,1
298,14,100,78,0,36.6,0.412,46,1
455,14,175,62,1,33.6,0.212,38,1
64,7,1412,66,1,32.8,0.258,42,1
...,...,...,...,...,...,...,...,...
593,2,82,52,1,28.5,1.699,25,0
621,2,92,76,1,24.2,1.698,28,0
123,5,132,80,0,26.8,0.186,69,0
459,9,134,74,0,25.9,0.460,81,0


In [10]:
from sklearn.ensemble import IsolationForest

def detect_outliers_isolation_forest(df, contamination=0.05):
    # ניקח רק עמודות מספריות
    X = df.select_dtypes(include=[np.number])

    # בניית המודל
    model = IsolationForest(contamination=contamination, random_state=42)

    # התאמת המודל לנתונים
    preds = model.fit_predict(X)

    # -1 אומר חריג, 1 אומר רגיל
    outliers = df[preds == -1]

    return outliers

# קריאה לפונקציה
outliers_iforest = detect_outliers_isolation_forest(df)

print("\n🔵 כל השורות שהוגדרו כחריגות לפי Isolation Forest:")
display(outliers_iforest)


🔵 כל השורות שהוגדרו כחריגות לפי Isolation Forest:


,Pregnancies,Glucose,BloodPressure,High Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
4,0,137,40,1,43.1,2.288,33,1
7,10,115,0,0,35.3,0.134,29,0
12,10,139,80,0,27.1,1.441,57,0
43,9,171,110,0,45.4,0.721,54,1
45,0,180,66,0,42.0,1.893,25,1
49,7,105,0,1,0.0,0.305,24,0
58,0,146,82,1,40.5,1.781,44,0
60,2,84,0,1,0.0,0.304,21,0
64,7,1412,66,1,32.8,0.258,42,1
78,0,131,0,0,43.2,0.270,26,1


In [11]:
# השוואת מספר החריגים בכל שיטה
print(f"🔵 מספר חריגים לפי סטיית תקן: {len(outliers_std)}")
print(f"🟠 מספר חריגים לפי Isolation Forest: {len(outliers_iforest)}")

# בדיקת חפיפה: אילו שורות הופיעו גם וגם
common_outliers = pd.merge(outliers_std, outliers_iforest, how='inner')
print(f"\n🟢 מספר חריגים משותפים בין שתי השיטות: {len(common_outliers)}")

# תצוגת השורות המשותפות
display(common_outliers)

🔵 מספר חריגים לפי סטיית תקן: 63
🟠 מספר חריגים לפי Isolation Forest: 39

🟢 מספר חריגים משותפים בין שתי השיטות: 31


,Pregnancies,Glucose,BloodPressure,High Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,7,1412,66,1,32.8,0.258,42,1
1,10,115,0,0,35.3,0.134,29,0
2,7,105,0,1,0.0,0.305,24,0
3,2,84,0,1,0.0,0.304,21,0
4,0,131,0,0,43.2,0.270,26,1
5,2,74,0,0,0.0,0.102,22,0
6,11,135,0,0,52.3,0.578,40,1
7,3,141,0,0,30.0,0.761,27,1
8,0,138,0,1,36.3,0.933,25,1
9,0,167,0,1,32.3,0.839,30,1


In [19]:
df_yes = df[df['Outcome'] == 1]
df_no = df[df['Outcome'] == 0]

p_yes = len(df_yes) / len(df)
p_no = len(df_no) / len(df)

means_yes = df_yes.mean()
stds_yes = df_yes.std()

means_no = df_no.mean()
stds_no = df_no.std()

sample = {
    'Pregnancies': 3,
    'Glucose': 100,
    'BloodPressure': 76,
    'High Insulin': 0,
    'BMI': 30,
    'DiabetesPedigreeFunction': 0.3,
    'Age': 32
}

import numpy as np

def gaussian_probability(x, mean, std):
    exponent = np.exp(-((x - mean) ** 2) / (2 * (std ** 2)))
    probability = (1 / (np.sqrt(2 * np.pi) * std)) * exponent
    return probability

likelihood_yes = p_yes
likelihood_no = p_no

for feature, value in sample.items():
    prob_yes = gaussian_probability(value, means_yes[feature], stds_yes[feature])
    prob_no = gaussian_probability(value, means_no[feature], stds_no[feature])

    likelihood_yes *= prob_yes
    likelihood_no *= prob_no

if likelihood_yes > likelihood_no:
    prediction = 1  # צפויה להיכנס להריון
else:
    prediction = 0  # לא צפויה להיכנס להריון

print(f"🔵 תחזית: {'נכנסה להריון' if prediction == 1 else 'לא נכנסה להריון'}")

🔵 תחזית: לא נכנסה להריון
